# Quick Start: The Ergodic Insurance Advantage

## Overview
- **What this notebook does:** Demonstrates the core insight of the framework in under 5 minutes -- why insurance that *appears* expensive actually *enhances* long-term growth.
- **Prerequisites:** [getting-started/01_setup_verification.ipynb](01_setup_verification.ipynb)
- **Estimated runtime:** < 2 minutes
- **Audience:** [Executive]

In [ ]:
"""Google Colab setup: mount Drive and install package dependencies.

Run this cell first. If prompted to restart the runtime, do so, then re-run all cells.
This cell is a no-op when running locally.
"""
import sys, os
if 'google.colab' in sys.modules:
    from google.colab import drive
    drive.mount('/content/drive')

    NOTEBOOK_DIR = '/content/drive/My Drive/Colab Notebooks/ei_notebooks/getting-started'

    os.chdir(NOTEBOOK_DIR)
    if NOTEBOOK_DIR not in sys.path:
        sys.path.append(NOTEBOOK_DIR)

    !pip install git+https://github.com/AlexFiliakov/Ergodic-Insurance-Limits.git -q 2>&1 | tail -3
    print('\nSetup complete. If you see numpy/scipy import errors below,')
    print('restart the runtime (Runtime > Restart runtime) and re-run all cells.')

## Setup

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

from ergodic_insurance import run_analysis

np.random.seed(42)

## 1. One-Call Analysis

`run_analysis` is the single entry point for the entire framework. Provide your company's basic financials and insurance structure, and it runs Monte Carlo simulations comparing insured vs. uninsured outcomes.

In [ ]:
results = run_analysis(
    initial_assets=10_000_000,       # $10M starting assets
    loss_frequency=2.5,              # ~2.5 loss events per year
    loss_severity_mean=1_000_000,    # Average loss of $1M
    deductible=500_000,              # $500K deductible
    coverage_limit=10_000_000,       # $10M coverage limit
    premium_rate=0.025,              # 2.5% premium rate
)

print(results.summary())

## 2. The Key Insight: Ensemble vs. Time Average

Traditional analysis uses the **ensemble average** (expected value across many parallel scenarios). But each business lives one trajectory over time. The **time average** captures the growth rate that a single entity actually experiences.

For multiplicative processes like wealth growth, these two averages diverge. Insurance closes that gap by reducing volatility.

In [ ]:
# Extract metrics from simulation results
insured_survival = sum(1 for r in results.insured_results if r.insolvency_year is None) / len(results.insured_results)
uninsured_survival = sum(1 for r in results.uninsured_results if r.insolvency_year is None) / len(results.uninsured_results)

insured_tw_roe = np.mean([r.calculate_time_weighted_roe() for r in results.insured_results])
uninsured_tw_roe = np.mean([r.calculate_time_weighted_roe() for r in results.uninsured_results])

insured_median_equity = np.median([float(r.equity[-1]) for r in results.insured_results])
uninsured_median_equity = np.median([float(r.equity[-1]) for r in results.uninsured_results])

print("Insured vs. Uninsured Outcomes")
print("=" * 55)
print(f"{'Metric':<30} {'Insured':>12} {'Uninsured':>12}")
print("-" * 55)
print(f"{'Survival rate':<30} {insured_survival * 100:>11.1f}% {uninsured_survival * 100:>11.1f}%")
print(f"{'Time-weighted ROE':<30} {insured_tw_roe * 100:>11.2f}% {uninsured_tw_roe * 100:>11.2f}%")
print(f"{'Median final equity':<30} ${insured_median_equity:>10,.0f} ${uninsured_median_equity:>10,.0f}")

## 3. Visualize the Ergodic Advantage

The plot below compares growth paths for insured (blue) and uninsured (red) scenarios. Notice how insurance narrows the spread of outcomes and protects against ruin.

In [ ]:
results.plot()

## 4. Why "Expensive" Insurance is Actually Cheap

Insurance premiums typically exceed expected losses by 50--200%. The traditional view labels this as "expensive." But the time-average perspective reveals a different story: the premium buys *growth stability*, which compounds over time into substantially more wealth.

In [ ]:
# Compare compounded outcomes
initial_equity = float(results.config.manufacturer.initial_assets)

print("Compounded Growth Comparison")
print("=" * 50)
print(f"  Initial equity:                ${initial_equity:,.0f}")
print(f"  Insured median final equity:   ${insured_median_equity:,.0f}")
print(f"  Uninsured median final equity: ${uninsured_median_equity:,.0f}")
if uninsured_median_equity > 0 and insured_median_equity > 0:
    advantage = (insured_median_equity / uninsured_median_equity - 1) * 100
    print(f"\n  Insurance wealth advantage:    {advantage:+.1f}%")
    print(f"  Survival rate advantage:       {(insured_survival - uninsured_survival) * 100:+.1f} percentage points")

## Key Takeaways

- **Ensemble average overstates returns.** It averages across many parallel worlds; you only live in one.
- **Time average is what you experience.** Volatility drag from large losses permanently impairs compounding.
- **Insurance is a growth tool, not just a cost.** By smoothing out catastrophic losses, insurance enhances your realized long-term growth rate.
- **The "premium puzzle" is resolved.** Rational actors buy insurance because they (correctly) optimize for time-average growth, not ensemble-average expectations.

## Next Steps

- **Build a model from scratch:** [getting-started/03_basic_manufacturer.ipynb](03_basic_manufacturer.ipynb)
- **Understand loss distributions:** [core/01_loss_distributions.ipynb](../core/01_loss_distributions.ipynb)
- **Deep dive into the ergodic advantage:** [core/03_ergodic_advantage.ipynb](../core/03_ergodic_advantage.ipynb)
- **Run Monte Carlo simulations:** [core/04_monte_carlo_simulation.ipynb](../core/04_monte_carlo_simulation.ipynb)